# ROMS Ocean Model Example

In [ ]:
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
import cmocean

import xarray as xr

Load a sample ROMS file. This is a subset of a full model available at 

    http://barataria.tamu.edu/thredds/catalog.html?dataset=txla_hindcast_agg
    
The subsetting was done using the following command on one of the output files:

    ds = xr.open_dataset('/d2/shared/TXLA_ROMS/output_20yr_obc/2001/ocean_his_0015.nc')
    ds.isel(ocean_time=slice(47, None, 3*24), xi_rho=slice(200, None), xi_u=slice(200, None),
            xi_v=slice(200, None), xi_psi=slice(200, None)).to_netcdf('ROMS_example.nc')

The `ROMS_example.nc` file contains a subset of the grid, and five time steps.

### How to load in a ROMS netcdf file in a way consistent with xarray and xgcm

In [ ]:
# load in the file
ds = xr.open_dataset('ROMS_example.nc', chunks={'ocean_time': 1})

# This is a way to turn on chunking and lazy evaluation. Opening with mfdataset, or 
# setting the chunking in the open_dataset would also achive this.
ds

### Add a lazilly calculated vertical coordinates

Write equations to calculate the vertical coordinate. These will be only evaluated when data is requested.

In [ ]:
if ds.Vtransform == 1:
    Zo_rho = ds.hc * (ds.s_rho - ds.Cs_r) + ds.Cs_r * ds.h
    z_rho = Zo_rho + ds.zeta * (1 + Zo_rho/ds.h)
#     Zo_w = ds.hc * (ds.s_w - ds.Cs_w) + ds.Cs_w * ds.h
#     z_w = Zo_w + ds.zeta * (1 + Zo_w/ds.h)
elif ds.Vtransform == 2:
    Zo_rho = (ds.hc * ds.s_rho + ds.Cs_r * ds.h) / (ds.hc + ds.h)
    z_rho = ds.zeta + (ds.zeta + ds.h) * Zo_rho
#     Zo_w = (ds.hc * ds.s_w + ds.Cs_w * ds.h) / (ds.hc + ds.h)
#     z_w = ds.zeta + (ds.zeta + ds.h) * Zo_w

ds.coords['z_rho'] = z_rho.transpose()   # needing transpose seems to be an xarray bug
# ds.coords['z_w'] = z_w.transpose()
ds.salt

### A naive vertical slice

Create a slice using the s-coordinate as the vertical dimension is typically not very informative.

In [ ]:
ds.salt.isel(xi_rho=50, ocean_time=0).plot()

In [ ]:
section = ds.salt.isel(xi_rho=50, eta_rho=slice(0, 167), ocean_time=0)
section.plot(x='lon_rho', y='z_rho', figsize=(15, 6), cmap=cmocean.cm.haline, clim=(25, 35))
plt.ylim([-100, 1]);

### A plan view

Now make a naive plan view, without any projection information, just using lon/lat as x/y.

In [ ]:
ds.salt.isel(s_rho=-1, ocean_time=0).plot(x='lon_rho', y='lat_rho')

And let's use a projection to make it nicer, and add a coast.

In [ ]:
proj = ccrs.LambertConformal(central_longitude=-92, central_latitude=29)
fig = plt.figure(figsize=(15, 5))
ax = plt.axes(projection=proj)
ds.salt.isel(s_rho=-1, ocean_time=0).plot(x='lon_rho', y='lat_rho', 
                                          cmap=cmocean.cm.haline, transform=ccrs.PlateCarree())

coast_10m = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='k', facecolor='0.8')
ax.add_feature(coast_10m)